In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
os.chdir(os.path.abspath('..'))

In [ ]:

classifier = MLkNN(k=3)


In [ ]:
""" test multilabel knn"""
import torch
from dinov2.eval.setup import setup_logger
from dinov2.data.loaders import make_dataset
from omegaconf import OmegaConf
from torch.utils.data import DataLoader
from skmultilearn.adapt import MLkNN

cfg = OmegaConf.create({
    'id': 'geobench.m-bigearthnet',
    'split': 'val'
})
global logger
logger = setup_logger('dinov2', to_sysout=True)

ds = make_dataset(cfg)
dl = DataLoader(ds, batch_size=10, num_workers=0, shuffle=False)


nsamples = 20
X_train = torch.stack([ds[i][0]['imgs'] for i in range(nsamples)])
y_train = torch.stack([ds[i][1] for i in range(nsamples)])
print('Built samples')

X_train = X_train.cpu().numpy()
y_train = y_train.cpu().numpy()
X_train = X_train.mean(axis=(1,3))
print(X_train.shape, y_train.shape)

classifier = MLkNN(k=3)
classifier.fit(X_train, y_train.astype(int))

# predict
predictions = classifier.predict(X_train)

# acc
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_train, predictions)
print('Accuracy:', acc)

In [ ]:
import torch
import re

In [ ]:
ckpt = torch.load('/data/panopticon/logs/dino_logs/dino_rgb/bl_dino_rgb/model_final.pth', map_location='cpu')

In [ ]:
pat = 'teacher.dino_head'

keys = [k for k in ckpt['model'].keys() if re.search(pat, k)]
print(keys)

In [ ]:
ckpt = torch.load('/data/panopticon/other_model_ckpts/dinov2/dinov2_vitb14_pretrain.pth', map_location='cpu')

In [ ]:

pat = 'patch_'
keys = [k for k in ckpt.keys() if re.search(pat, k)]
print(keys)

In [ ]:
print(ckpt['patch_embed.proj.weight'].shape)
print(ckpt['patch_embed.proj.bias'].shape)

In [ ]:
new_weights = ckpt['patch_embed.proj.weight'].mean(axis=1).unsqueeze(1).unsqueeze(1)
new_weights = {
    'rgbmean_patch_embed.proj.weight': new_weights,
    'rgbmean_patch_embed.proj.bias': ckpt['patch_embed.proj.bias']
}
for k,v in new_weights.items():
    print(k, v.shape)
torch.save(new_weights, '/data/panopticon/other_model_ckpts/dinov2/dinov2_vitb14_pretrain_pe_rgbmean.pth')

In [ ]:
from dinov2.utils.utils import load_pretrained_weights
from dinov2.models.panopticon import ChnWisePatchEmb


our_pe = ChnWisePatchEmb(embed_dim=768, patch_size=14)

pretrained_weights = [dict(
    path = '/data/panopticon/other_model_ckpts/dinov2/dinov2_vitb14_pretrain_pe_rgbmean.pth',
    prefix_map = {'rgbmean_patch_embed': ''}
)]

load_pretrained_weights(our_pe, pretrained_weights=pretrained_weights)

In [ ]:
for n,p in our_pe.named_parameters():
    print(n, p.shape)

In [ ]:
import torch

torch.arange(10)[[1,4,5]]

In [ ]:
from omegaconf import OmegaConf
from dinov2.data.loaders import make_dataset   
from dinov2.eval.setup import setup_logger

global logger
logger = setup_logger('dinov2', to_sysout=True)

cfg = OmegaConf.create({
    'id': 'FmowDataset',
    'root': '/data/panopticon/datasets/',
    'split': 'fmow/metadata_v2/fmow_iwm_onid_train_val.parquet'
})
dino_augm_cfg = OmegaConf.create({
    'id': 'ChnSpatialAugmentationV2',
    'global_crops_scale': [0.32, 1.0],
    'local_crops_number': 4,
    'global_crops_number': 2,
    'local_crops_scale': [0.05, 0.32],
    'global_crops_size': 224,
    'local_crops_size': 98,
    'global_crops_spectral_size': [6,13],
    'local_crops_spectral_size': [3,6],
    'global_modes_probs': [0.8, 0.1, 0.1],
    'local_modes_probs': [0.2, 0.8],
})

ds = make_dataset(cfg, dino_augm_cfg)

In [ ]:
from dinov2.data.loaders import make_dataset  
from omegaconf import OmegaConf
from dinov2.eval.setup import setup_logger

global logger
logger = setup_logger('dinov2', to_sysout=True)

cfg = OmegaConf.load('/home/lewaldm/code/PanOpticOn/dinov2/configs/debug.yaml')
ds = make_dataset(cfg.train.dataset,cfg.train.dino_augm)
ds[0]